# Set connection with GitHub

In [1]:
# ! cd /kaggle/working && rm -rf RECsys_Challenge2024

In [2]:
from kaggle_secrets import UserSecretsClient

token = UserSecretsClient().get_secret("Token")

! git clone https://{token}@github.com/madratak/RECsys_Challenge2024.git

Cloning into 'RECsys_Challenge2024'...
remote: Enumerating objects: 3871, done.
remote: Counting objects: 100% (1344/1344), done.
remote: Compressing objects: 100% (550/550), done.
remote: Total 3871 (delta 737), reused 1116 (delta 606), pack-reused 2527 (from 1)
Receiving objects: 100% (3871/3871), 165.12 MiB | 21.58 MiB/s, done.
Resolving deltas: 100% (2256/2256), done.
Updating files: 100% (370/370), done.


In [3]:
! pip install PyGithub requests

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 375.9/375.9 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 856.7/856.7 kB 23.7 MB/s eta 0:00:00


In [4]:
%cd /kaggle/working/RECsys_Challenge2024
! python run_compile_all_cython.py

/kaggle/working/RECsys_Challenge2024
run_compile_all_cython: Found 11 Cython files in 5 folders...
run_compile_all_cython: All files will be compiled using your current python environment: '/opt/conda/bin/python'
Compiling [1/11]: MatrixFactorization_Cython_Epoch.pyx... 
In file included from /opt/conda/lib/python3.10/site-packages/numpy/core/include/numpy/ndarraytypes.h:1929,
                 from /opt/conda/lib/python3.10/site-packages/numpy/core/include/numpy/ndarrayobject.h:12,
                 from /opt/conda/lib/python3.10/site-packages/numpy/core/include/numpy/arrayobject.h:5,
                 from MatrixFactorization_Cython_Epoch.c:1252:
/opt/conda/lib/python3.10/site-packages/numpy/core/include/numpy/npy_1_7_deprecated_api.h:17:2: warning: #warning "Using deprecated NumPy API, disable it with " "#define NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION" [-Wcpp]
   17 | #warning "Using deprecated NumPy API, disable it with " \
      |  ^~~~~~~
MatrixFactorization_Cython_Epoch.c: In fun

In [5]:
from Utils.notebookFunctions import *
import numpy as np
import pandas as pd
import scipy.sparse as sps
import time
import shutil
import optuna
import json
import os
from Utils.seconds_to_biggest_unit import seconds_to_biggest_unit


K_PATH = '/kaggle/working/RECsys_Challenge2024'
GH_PATH = 'TrainedModels/WithoutKFCV'
D_PATH = 'Hybrid/LinearCombination'

np.random.seed(42)

## Import the repository

In [6]:
repo = get_repo_from_github(token)

Repository 'RECsys_Challenge2024' found.


In [7]:
config = {
    'model': 'SecondLinearCombination',
    'tune_parameters': True,
    'database_path': '/kaggle/working/history_SecondLinearCombination.db',
    'copy_prev_best_params': False,
    'tune_best_params': True,
    'save_github': True
}

Import the database where previous tuning trials have been saved.

In [8]:
try:
    shutil.copyfile(
        f'{K_PATH}/{GH_PATH}/{D_PATH}/{config["model"]}Recommender/history_{config["model"]}.db', 
        config['database_path']
    )
except FileNotFoundError:
    pass # if not present optuna will create it

# Construction of URM and ICM matrices

In [9]:
URM_all_dataframe = pd.read_csv("/kaggle/input/recommender-system-2024-challenge-polimi/data_train.csv")

n_users = len(URM_all_dataframe["user_id"].unique())
n_items = len(URM_all_dataframe["item_id"].unique())

URM_all = sps.csr_matrix((URM_all_dataframe["data"].values, 
                          (URM_all_dataframe["user_id"].values, URM_all_dataframe["item_id"].values)),
                        shape = (n_users, n_items))

URM_all

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 1764607 stored elements and shape (35736, 38121)>

In [10]:
ICM_dataframe = pd.read_csv("/kaggle/input/recommender-system-2024-challenge-polimi/data_ICM_metadata.csv")

n_items = len(ICM_dataframe["item_id"].unique())
n_features = len(ICM_dataframe["feature_id"].unique())

ICM_all = sps.csr_matrix((ICM_dataframe["data"].values, 
                          (ICM_dataframe["item_id"].values, ICM_dataframe["feature_id"].values)),
                        shape = (n_items, n_features))

ICM_all

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 2940040 stored elements and shape (38121, 94331)>

# Training

In [11]:
from Evaluation.Evaluator import EvaluatorHoldout
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample

URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.80)

evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])

EvaluatorHoldout: Ignoring 141 ( 0.4%) Users that have less than 1 test interactions


## Import the best recommeders previously trained.

In [12]:
from Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from Recommenders.SLIM.SLIMElasticNetRecommender import SLIMElasticNetRecommender
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender

recommenders = {
    "ItemKNNCF": ItemKNNCFRecommender,
    "SLIMElasticNet": SLIMElasticNetRecommender,
    "RP3beta": RP3betaRecommender
}

paths_to_best_params = {
    "ItemKNNCF": "KNN",
    "SLIMElasticNet": "SLIM",
    "RP3beta": "GraphBased"
}


loaded_recommenders = {}

for recommender_name, recommender_class in recommenders.items():
    
    start_time = time.time()
    
    print(f"{recommender_name} Model - TRAINING with its best parameters.")
    try:
        recommender = recommender_class(URM_train)
    except Exception as e:
        recommender = recommender_class(URM_train, ICM_all)

    # Extract best parameter values of the relative recommender model.
    with open(f'{GH_PATH}/{paths_to_best_params[recommender_name]}/{recommender_name}Recommender/best_params_{recommender_name}.json', 'r') as best_params_json:
            best_params = json.load(best_params_json)
    
    recommender.fit(**best_params)
    
    loaded_recommenders[recommender_name] = recommender
    
    new_time_value, new_time_unit = seconds_to_biggest_unit(time.time() - start_time)
    print("Training done in {:2f}{}.\n".format(new_time_value, new_time_unit))

ItemKNNCF Model - TRAINING with its best parameters.
Similarity column 38121 (100.0%), 2740.83 column/sec. Elapsed time 13.91 sec
Training done in 14.119500sec.

SLIMElasticNet Model - TRAINING with its best parameters.
SLIMElasticNetRecommender: Processed 3181 ( 8.3%) in 5.00 min. Items per second: 10.60
SLIMElasticNetRecommender: Processed 6320 (16.6%) in 10.00 min. Items per second: 10.53
SLIMElasticNetRecommender: Processed 9527 (25.0%) in 15.00 min. Items per second: 10.58
SLIMElasticNetRecommender: Processed 12601 (33.1%) in 20.00 min. Items per second: 10.50
SLIMElasticNetRecommender: Processed 15727 (41.3%) in 25.00 min. Items per second: 10.48
SLIMElasticNetRecommender: Processed 18737 (49.2%) in 30.01 min. Items per second: 10.41
SLIMElasticNetRecommender: Processed 21829 (57.3%) in 35.01 min. Items per second: 10.39
SLIMElasticNetRecommender: Processed 24900 (65.3%) in 40.01 min. Items per second: 10.37
SLIMElasticNetRecommender: Processed 28037 (73.5%) in 45.01 min. Items p

In [13]:
from Recommenders.Hybrid.LinearCombinationRecommender import LinearCombinationRecommender 

def objective_function_(optuna_trial):
    
    recommender_instance = LinearCombinationRecommender(URM_train, loaded_recommenders.values())
    
    norm = optuna_trial.suggest_categorical('norm', [1, 2, np.inf, -np.inf])
    itemknn_weight = optuna_trial.suggest_float('itemknn_weight', 0.0, 1.0)
    slim_weight = optuna_trial.suggest_float('slim_weight', 0.0, 1.0)
    rp3beta_weight = optuna_trial.suggest_float('rp3beta_weight', 0.0, 1.0)

    # Ensure the weights sum to 1 (Normalization)
    total_weight = itemknn_weight + slim_weight + rp3beta_weight
    itemknn_weight /= total_weight
    slim_weight /= total_weight
    rp3beta_weight /= total_weight
    
    full_hyperp = {
                    "weights": [itemknn_weight, slim_weight,rp3beta_weight],
                    "norm": norm
                  }        
    
    recommender_instance.fit(**full_hyperp)
    
    result_df, _ = evaluator_validation.evaluateRecommender(recommender_instance)
    
    return result_df.loc[10]["MAP"]

In [14]:
if config['tune_parameters']:
    
    optuna_study = optuna.create_study(direction='maximize', study_name=f'hyperparameters_tuning_{config["model"]}',
                                storage=f'sqlite:///{config["database_path"]}', load_if_exists=True)

    optuna_study.optimize(objective_function_, n_trials=50)

[I 2024-12-03 10:47:33,744] Using an existing study with name 'hyperparameters_tuning_SecondLinearCombination' instead of creating a new one.


LinearCombinationRecommender: Fit completed in 0.00 seconds.
EvaluatorHoldout: Processed 35595 (100.0%) in 1.52 min. Users per second: 391


[I 2024-12-03 10:49:05,065] Trial 150 finished with value: 0.06002877388254337 and parameters: {'norm': 1, 'itemknn_weight': 0.09904412832781548, 'slim_weight': 0.9453934401162589, 'rp3beta_weight': 0.1938288686637028}. Best is trial 124 with value: 0.06076377879896466.


LinearCombinationRecommender: Fit completed in 0.00 seconds.
EvaluatorHoldout: Processed 35595 (100.0%) in 1.52 min. Users per second: 391


[I 2024-12-03 10:50:36,394] Trial 151 finished with value: 0.060733739579081195 and parameters: {'norm': 2, 'itemknn_weight': 0.034038118978174624, 'slim_weight': 0.91063167856302, 'rp3beta_weight': 0.1714334855006142}. Best is trial 124 with value: 0.06076377879896466.


...

[I 2024-12-03 12:00:34,167] Trial 197 finished with value: 0.06068943270523388 and parameters: {'norm': 2, 'itemknn_weight': 0.04173099887865642, 'slim_weight': 0.792052170833838, 'rp3beta_weight': 0.0683830708318529}. Best is trial 124 with value: 0.06076377879896466.


LinearCombinationRecommender: Fit completed in 0.00 seconds.
EvaluatorHoldout: Processed 35595 (100.0%) in 1.52 min. Users per second: 391


[I 2024-12-03 12:02:05,304] Trial 198 finished with value: 0.06074403061781734 and parameters: {'norm': 2, 'itemknn_weight': 0.05707202250065098, 'slim_weight': 0.8209709622164788, 'rp3beta_weight': 0.09923699895719275}. Best is trial 124 with value: 0.06076377879896466.


LinearCombinationRecommender: Fit completed in 0.00 seconds.
EvaluatorHoldout: Processed 35595 (100.0%) in 1.52 min. Users per second: 391


[I 2024-12-03 12:03:36,497] Trial 199 finished with value: 0.06063713581138604 and parameters: {'norm': 2, 'itemknn_weight': 0.03476067908623501, 'slim_weight': 0.7653418749690175, 'rp3beta_weight': 0.038939999578766066}. Best is trial 124 with value: 0.06076377879896466.


## Some optuna visualizations on recommender parameters

In [15]:
if not config['tune_parameters']:
    optuna_study = optuna.load_study(study_name=f'hyperparameters_tuning_{config["model"]}', storage=f'sqlite:///{config["database_path"]}')
    
fig = optuna.visualization.plot_slice(optuna_study)
fig.show()

In [16]:
if not config['tune_parameters']:
    optuna_study = optuna.load_study(study_name=f'hyperparameters_tuning_{config["model"]}', storage=f'sqlite:///{config["database_path"]}')
    
fig = optuna.visualization.plot_param_importances(optuna_study)
fig.show()

## Let's train the recommender with best parameter values

In [17]:
if config['tune_best_params']:
    
    loaded_recommenders = {}
    
    for recommender_name, recommender_class in recommenders.items():
        
        start_time = time.time()
        
        print(f"{recommender_name} Model - TRAINING with its best parameters.")
        try:
            recommender = recommender_class(URM_train + URM_validation)
        except Exception as e:
            recommender = recommender_class(URM_train + URM_validation, ICM_all)
    
        # Extract best parameter values of the relative recommender model.
        with open(f'{GH_PATH}/{paths_to_best_params[recommender_name]}/{recommender_name}Recommender/best_params_{recommender_name}.json', 'r') as best_params_json:
                best_params = json.load(best_params_json)
        
        recommender.fit(**best_params)
        
        loaded_recommenders[recommender_name] = recommender
        
        new_time_value, new_time_unit = seconds_to_biggest_unit(time.time() - start_time)
        print("Training done in {:2f}{}.\n".format(new_time_value, new_time_unit))

ItemKNNCF Model - TRAINING with its best parameters.
Similarity column 38121 (100.0%), 2546.14 column/sec. Elapsed time 14.97 sec
Training done in 15.232244sec.

SLIMElasticNet Model - TRAINING with its best parameters.
SLIMElasticNetRecommender: Processed 2512 ( 6.6%) in 5.00 min. Items per second: 8.37
SLIMElasticNetRecommender: Processed 5028 (13.2%) in 10.00 min. Items per second: 8.38
SLIMElasticNetRecommender: Processed 7489 (19.6%) in 15.00 min. Items per second: 8.32
SLIMElasticNetRecommender: Processed 9900 (26.0%) in 20.00 min. Items per second: 8.25
SLIMElasticNetRecommender: Processed 12276 (32.2%) in 25.00 min. Items per second: 8.18
SLIMElasticNetRecommender: Processed 14759 (38.7%) in 30.01 min. Items per second: 8.20
SLIMElasticNetRecommender: Processed 17249 (45.2%) in 35.01 min. Items per second: 8.21
SLIMElasticNetRecommender: Processed 19736 (51.8%) in 40.01 min. Items per second: 8.22
SLIMElasticNetRecommender: Processed 22207 (58.3%) in 45.01 min. Items per second

In [18]:
if config['tune_best_params']:

    if config['tune_parameters']:
        best_params = optuna_study.best_trial.params
    else: 
        with open(f'{K_PATH}/{GH_PATH}/{D_PATH}/{config["model"]}Recommender/best_params_{config["model"]}.json', 'r') as best_params_json:
            best_params = json.load(best_params_json)
            
    fit_params = {
    "weights": list(best_params.values())[1:], 
    "norm": best_params["norm"]
    }
    
    recommender_instance = LinearCombinationRecommender(URM_train + URM_validation, loaded_recommenders.values())
    recommender_instance.fit(**fit_params)

LinearCombinationRecommender: Fit completed in 0.00 seconds.


# Testing

Create the recommendations for the submission. 

In [19]:
if config['tune_best_params']:

    data_target_users_test = pd.read_csv('/kaggle/input/recommender-system-2024-challenge-polimi/data_target_users_test.csv')
    create_submission(data_target_users_test, recommender_instance, f'/kaggle/working/submission_{config["model"]}.csv')

Submission file saved as /kaggle/working/submission_SecondLinearCombination.csv


# Save Version on GitHub 

Write or import a json file where best hyperparameters are saved. 

In [20]:
if config['tune_parameters']:
    with open(f'/kaggle/working/best_params_{config["model"]}.json', 'w') as params_file:
        json.dump(optuna_study.best_params, params_file)
        
    if config['save_github']:
        upload_file(
            f'/kaggle/working/best_params_{config["model"]}.json', 
            f'{GH_PATH}/{D_PATH}/{config["model"]}Recommender/best_params_{config["model"]}.json', 
            f'{config["model"]} tuning results (from kaggle notebook)',
            repo
        )
elif config['copy_prev_best_params']:
    shutil.copyfile(
        f'{K_PATH}/{GH_PATH}/{D_PATH}/{config["model"]}Recommender/'\
        f'best_params_{config["model"]}.json', 
        f'/kaggle/working/best_params_{config["model"]}.json'
    )

File 'TrainedModels/WithoutKFCV/Hybrid/LinearCombination/SecondLinearCombinationRecommender/best_params_SecondLinearCombination.json' updated successfully.


Save the history of the tuned model.

In [21]:
if config['save_github'] and config['tune_parameters']:
    upload_file(
        config['database_path'], 
        f'{GH_PATH}/{D_PATH}/{config["model"]}Recommender/history_{config["model"]}.db',
        f'Tuning {config["model"]} db updated results (from kaggle notebook)',
        repo
    )

File 'TrainedModels/WithoutKFCV/Hybrid/LinearCombination/SecondLinearCombinationRecommender/history_SecondLinearCombination.db' updated successfully.


Save the best trained model and its submission.

In [22]:
if config['save_github'] and config['tune_best_params']: 
    upload_file(
                f'/kaggle/working/submission_{config["model"]}.csv', 
                f'{GH_PATH}/{D_PATH}/{config["model"]}Recommender/Submission/submission_{config["model"]}.csv', 
                f'New {config["model"]} submission (from kaggle notebook)',
                repo
            )

File 'TrainedModels/WithoutKFCV/Hybrid/LinearCombination/SecondLinearCombinationRecommender/Submission/submission_SecondLinearCombination.csv' created successfully.
